**Script de recherche et d'appariement des données Geod'Air**

**Description**

- Récupération automatique des données des 10 stations les plus proches, quelle que soit la typologie de la station (de fond, de proximité, etc.), sur la période d’enquête d’intérêt

- Récupération automatique des données des 3 stations de fond les plus proches, sur la période d’enquête d’intérêt

- Calcul des distances des stations aux bâtiments

- Qualification des données récupérées (indicateur(s) de qualité de la mesure, données manquantes, identification des valeurs aberrantes (ex : concentration < 0) )

- Construction d’une table structurée : identifiant du bâtiment, période d’enquête, longitude et latitude du bâtiment, identifiants des stations récupérées, longitudes et latitudes des stations, distances entre le bâtiment et les stations, typologies des stations (de fond, de proximité), caractère mobile ou fixe des stations, substances et concentrations mesurées, période de mesure, indicateur(s) de qualité des mesures, limites de détection et quantification.

In [1]:
# Automatic reload of the Jupyter Notebook
%load_ext autoreload
%autoreload 2

# Load the .env
%load_ext dotenv

# Load the API key

In [2]:
# Chargement de la clé API Geod'Air
import os

apikey_geodair = os.environ.get('apikey_geodair')

if apikey_geodair is not None:
    print('✅ apikey_geodair variable loaded')

else:
    print('❌ apikey_geodair variable not loaded')

✅ apikey_geodair variable loaded


# API - Stations

Cette API permet d'obtenir la liste des stations de mesure ainsi que toutes les informations les caractérisant (type de site, code, adresse, date de mise en service, etc.).

## Requête

Renseigner dans :

- **date** : la date souhaitée
- **apikey** : la clé API

In [3]:
# API GET request pour obtenir les données concernant les stations Geod'air
import requests

# Paramètres à choisir
date='2023-04-25' # Si utilisation de l'API pour obtenir les moyennes journalières, sinon inutile
apikey=apikey_geodair

url_stations = f'https://www.geodair.fr/api-ext/station/export?date={date}'
headers = {'accept': 'text/csv', 'apikey': apikey}

req = requests.get(url_stations, headers=headers)

In [4]:
# API Response headers
req.headers

{'Date': 'Tue, 23 May 2023 15:02:09 GMT', 'Content-Type': 'text/csv;charset=ISO-8859-1', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'RateLimit-Remaining': '14', 'RateLimit-Reset': '3471', 'X-RateLimit-Remaining-Hour': '14', 'X-RateLimit-Limit-Hour': '15', 'RateLimit-Limit': '15', 'X-Token-Expires-In': '35645.157455921', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers, Origin', 'Content-Disposition': 'attachment; filename="Stations_2023-04-25.csv"', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'X-Auth-Token', 'X-Kong-Upstream-Latency': '372', 'X-Kong-Proxy-Latency': '2', 'Via': 'kong/2.6.0'}

In [5]:
# API data raw
data_stations = req.content.decode('ISO-8859-1')

In [6]:
# API data into a Pandas DataFrame
import pandas as pd
import tempfile
import io

df_stations = pd.read_csv(io.StringIO(data_stations), sep=";")

In [7]:
#df_stations

In [8]:
# Ne garder que les informations essentielles concernant les stations Geod'air
df_stations_geodair = df_stations[[
    "Type de site",
    "Code",
    "Alias",
    "Longitude",
    "Latitude",
    "Implantation",
    "Mobilité"]]

df_stations_geodair.head(20)

,Type de site,Code,Alias,Longitude,Latitude,Implantation,Mobilité
0,classique,FR28028,Bethune Stade,2.635692,50.535385,Urbaine,Station Fixe
1,classique,FR38024,RDT,55.302844,-21.213212,Rurale près des villes,Station Fixe
2,classique,FR07056,NaN,4.226006,44.983770,Rurale régionale,Station Fixe
3,classique,FR02043,Marignane,5.222281,43.416531,Urbaine,Station Fixe
4,classique,FR34043,Oysonville,1.955611,48.391139,Rurale régionale,Station Fixe
5,classique,FR09003,place de VERDUN,-1.153603,46.162556,Urbaine,Station Fixe
6,classique,FR04331,LIMAY,1.754142,48.985523,Périurbaine,Station Fixe
7,classique,FR04100,VILLEMOMBLE,2.507222,48.881947,Urbaine,Station Fixe
8,classique,FR04063,MANTES-LA-JOLIE,1.703300,48.996400,Périurbaine,Station Fixe
9,classique,FR06134,Saint Amand,3.427940,50.456470,Périurbaine,Station Fixe


## Description

In [ ]:
# Df shape

df_stations.shape

In [ ]:
# Df info

df_stations.info()

In [ ]:
# Df Nb de null

df_stations.isnull().sum()

In [ ]:
# Df % de null

round(df_stations.isnull().sum()/len(df_stations)*100,1)

# Statistiques

Plusieurs statistiques peuvent être obtenues via les API Geod'Air :
- Moyenne journalière
- Moyenne horaire
- Moyenne annuelle
- Maximum horaire journalier

Pour chacun de ces fichiers, la valeur à prendre se trouve dans la colonne "**Valeur brute**" (la colonne "Valeur" est un arrondi à l'entier près).

Pour toutes ces API, 3 informations sont à renseigner :
- **date** : la date souhaitée
- **polluant** : le code FR du polluant (chaîne de 2 caractères)
- **apikey** : la clé API

Liste des polluants avec leurs codes :

Source : https://www.geodair.fr/sites/default/files/media-files/2022-05/liste_polluants.pdf

In [ ]:
from IPython.display import Image

fig = Image(filename=('Geodair - Liste polluants.png'))
fig

## API - Moyenne journalière

### Requête

In [ ]:
# API GET request

import requests

# Paramètres à choisir
date='2023-04-25'
polluant='03'
apikey=apikey_geodair

url_moy_j = f'https://www.geodair.fr/api-ext/MoyJ/export?date={date}&polluant={polluant}'
headers = {'accept': 'text/plain', 'apikey': apikey}

req = requests.get(url_moy_j, headers=headers)

In [ ]:
# API Response headers

req.headers

In [ ]:
# Obtenir l'ID du fichier à télécharger

id_fichier = req.content.decode('UTF-8')
id_fichier

In [ ]:
# Télécharger le fichier

url_download = f'https://www.geodair.fr/api-ext/download?id={id_fichier}'
headers = {'accept': 'application/octet-stream', 'apikey': apikey}

req_download = requests.get(url_download, headers=headers)

In [ ]:
# API Response headers

req_download.headers

In [ ]:
# API data raw

data_moy_j = req_download.content.decode('UTF-8')

In [ ]:
# API data into a Pandas DataFrame

df_moy_j = pd.read_csv(io.StringIO(data_moy_j), sep=";")

In [ ]:
df_moy_j

In [ ]:
df_moy_j.info()

In [ ]:
df_moy_j_short = df_moy_j[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]
df_moy_j_short.head()

## API - Moyenne horaire

In [ ]:
# API GET request

import requests

# Paramètres à choisir
date='2023-04-25'
polluant='03'
apikey=apikey_geodair

url_moy_h = f'https://www.geodair.fr/api-ext/MoyH/export?date={date}&polluant={polluant}'
headers = {'accept': 'text/plain', 'apikey': apikey}

req = requests.get(url_moy_h, headers=headers)

In [ ]:
# API Response headers

req.headers

In [ ]:
# Obtenir l'ID du fichier à télécharger

id_fichier = req.content.decode('UTF-8')
id_fichier

In [ ]:
# Télécharger le fichier

url_download = f'https://www.geodair.fr/api-ext/download?id={id_fichier}'
headers = {'accept': 'application/octet-stream', 'apikey': apikey}

req_download = requests.get(url_download, headers=headers)

In [ ]:
# API Response headers

req_download.headers

In [ ]:
# API data raw

data_moy_h = req_download.content.decode('UTF-8')

In [ ]:
# API data into a Pandas DataFrame

df_moy_h = pd.read_csv(io.StringIO(data_moy_h), sep=";")

In [ ]:
df_moy_h

# Bâtiments du CSTB

In [9]:
# Obtenir les données concernant les bâtiments CSTB
cstb_file = '../../data/cstb/API2BUILD_COORD_DATES_BATI.xlsx'
df_immeubles_cstb = pd.read_excel(cstb_file)

In [10]:
df_immeubles_cstb

,Identifiant,Longitude,Latitude,Date_debut_enquete,Date_fin_enquete
0,B001,5.365996,43.314652,2016-11-29,NaT
1,B002,5.389293,43.310217,2016-11-25,NaT
2,B003,5.369872,43.310963,2016-12-16,NaT
3,B004,5.379358,43.286702,2017-06-12,NaT
4,B005,5.041559,43.417900,2017-06-06,NaT
...,...,...,...,...,...
1101,L568,2.273457,48.825051,2022-11-08,2022-11-15
1102,L569,2.847896,49.990562,2022-09-02,2022-09-09
1103,L570,2.158504,48.940902,2022-09-01,2022-09-08
1104,L571,1.879346,48.757418,2022-04-04,2022-04-11


## Description

In [ ]:
# Df % de null

round(df_cstb.isnull().sum()/len(df_cstb)*100,1)

# Script

In [11]:
# Bâtiments CSTB
df_immeubles_cstb

,Identifiant,Longitude,Latitude,Date_debut_enquete,Date_fin_enquete
0,B001,5.365996,43.314652,2016-11-29,NaT
1,B002,5.389293,43.310217,2016-11-25,NaT
2,B003,5.369872,43.310963,2016-12-16,NaT
3,B004,5.379358,43.286702,2017-06-12,NaT
4,B005,5.041559,43.417900,2017-06-06,NaT
...,...,...,...,...,...
1101,L568,2.273457,48.825051,2022-11-08,2022-11-15
1102,L569,2.847896,49.990562,2022-09-02,2022-09-09
1103,L570,2.158504,48.940902,2022-09-01,2022-09-08
1104,L571,1.879346,48.757418,2022-04-04,2022-04-11


In [12]:
# Stations Geod'air
df_stations_geodair

,Type de site,Code,Alias,Longitude,Latitude,Implantation,Mobilité
0,classique,FR28028,Bethune Stade,2.635692,50.535385,Urbaine,Station Fixe
1,classique,FR38024,RDT,55.302844,-21.213212,Rurale près des villes,Station Fixe
2,classique,FR07056,NaN,4.226006,44.983770,Rurale régionale,Station Fixe
3,classique,FR02043,Marignane,5.222281,43.416531,Urbaine,Station Fixe
4,classique,FR34043,Oysonville,1.955611,48.391139,Rurale régionale,Station Fixe
...,...,...,...,...,...,...,...
559,classique,FR12047,BESSIERES ECONOTRE,1.595475,43.803139,Rurale près des villes,Station Fixe
560,classique,FR22009,Pays Naborien (Carling),6.716886,49.165726,Périurbaine,Station Fixe
561,classique,FR08018,Montpellier St Gely,3.800210,43.692600,Périurbaine,Station Fixe
562,classique,FR05074,Le Havre Ec. Herriot,0.100703,49.489630,Urbaine,Station Fixe


## 10 stations Geod'air les plus proches

In [13]:
# Créer la fonction pour calculer la distance haversine entre 2 points
from math import radians, cos, sin, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculer la distance haversine entre 2 points en kilomètres
    """
    
    R = 6371  # rayon de la Terre en km
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    delta_phi = radians(lat2 - lat1)
    delta_lambda = radians(lon2 - lon1)
    
    a = sin(delta_phi / 2) ** 2 + cos(phi1) * cos(phi2) * sin(delta_lambda / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    d = R * c
    
    return d

# Calculer la distance haversine de chaque immeuble CSTB avec toutes les stations Geod'air
distances_df = pd.DataFrame()

for _, immeuble in df_immeubles_cstb.iterrows():
    distances = []
    
    for _, station in df_stations_geodair.iterrows():
        distance = haversine(immeuble['Latitude'], immeuble['Longitude'],
                             station['Latitude'], station['Longitude'])
        distances.append(distance)
        
    distances_df[immeuble['Identifiant']] = distances


/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

...

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  distances_df[immeuble['Identifiant']] = distances
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/701541978.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [14]:
distances_df

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,L563,L564,L565,L566,L567,L568,L569,L570,L571,L572
0,829.093880,830.014747,829.566041,832.368348,812.084963,562.435227,560.994611,562.887761,440.412484,440.051074,...,196.023879,190.869341,185.833766,196.023879,62.431075,191.957305,62.431075,180.583252,205.060609,180.583252
1,8800.361714,8798.653598,8799.851916,8797.440350,8827.323985,9395.393012,9395.850725,9394.790756,9067.977144,9068.291468,...,9396.616127,9396.357034,9396.512456,9396.616127,9445.605209,9390.151597,9445.605209,9404.983847,9406.836362,9404.983847
2,206.680379,207.947588,207.186138,209.950058,185.855433,438.329133,438.201732,437.916649,286.704654,287.025283,...,453.658455,456.294726,459.203701,453.658455,566.250593,452.113604,566.250593,467.101501,455.897395,467.101501
3,16.226448,17.945259,16.737670,19.227852,14.597412,587.579643,587.795908,587.046523,430.538640,430.922866,...,644.648991,647.128000,649.876793,644.648991,752.981576,642.781356,752.981576,657.871668,647.259949,657.871668
4,623.055469,624.270703,623.557577,626.335229,602.216842,342.457166,341.106144,342.772003,327.633547,327.423363,...,47.551348,53.086159,58.587102,47.551348,189.296044,53.610057,189.296044,62.920434,41.113098,62.920434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,308.613813,310.559124,309.002744,310.302809,280.710643,338.739970,339.447390,338.083312,514.305849,514.557723,...,557.510595,562.312624,567.118127,557.510595,694.531631,560.826971,694.531631,572.919180,551.321378,572.919180
560,658.821857,659.030579,659.180802,661.735096,651.916627,677.913190,676.753546,678.051756,219.557376,219.184928,...,337.319201,332.426947,328.102469,337.319201,293.604164,326.337540,293.604164,333.068344,356.016295,333.068344
561,133.092108,135.035100,133.522272,135.148834,104.591403,476.033187,476.359162,475.467325,431.118577,431.458370,...,582.657313,586.038942,589.603775,582.657313,704.020743,582.613881,704.020743,597.008527,582.192099,597.008527
562,796.065152,797.400897,796.575481,799.311920,773.601447,383.237508,381.792133,383.810259,501.849710,501.596130,...,165.930128,167.714170,169.211736,165.930128,205.108231,174.420233,205.108231,161.434980,152.902954,161.434980


In [15]:
# Pour chaque bâtiment CSTB, obtenir les 10 stations Geod'air les plus proches
# Stations Geod'air classées de la plus proche à la plus lointaine
closest10_stations_df_code = pd.DataFrame()
closest10_stations_df_km = pd.DataFrame()

for immeuble_cstb in df_immeubles_cstb['Identifiant']:
    distances = distances_df[immeuble_cstb].sort_values()
    
    closest10_stations_code = []
    closest10_stations_km = []
    
    for i in range(10):
        geodair_station_id = df_stations_geodair.loc[distances.index[i], 'Code']
        closest10_stations_code.append(geodair_station_id)
        
        closest10_stations_km.append(distances.iloc[i])
        
    closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
    closest10_stations_df_km[immeuble_cstb] = closest10_stations_km

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_code[immeuble_cstb] = closest10_stations_code
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closest10_stations_df_km[immeuble_cstb] = closest10_stations_km
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/377217502.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [16]:
closest10_stations_df_code

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,L563,L564,L565,L566,L567,L568,L569,L570,L571,L572
0,FR03043,FR03043,FR03043,FR03006,FR02004,FR09003,FR09003,FR09003,FR82006,FR82006,...,FR04029,FR04179,FR04150,FR04029,FR18045,FR04179,FR18045,FR04051,FR04181,FR04051
1,FR03014,FR03006,FR03014,FR03043,FR02028,FR09008,FR09008,FR09008,FR82001,FR82001,...,FR04179,FR04029,FR04017,FR04179,FR18042,FR04060,FR18042,FR04150,FR04029,FR04150
2,FR03006,FR03014,FR03006,FR03014,FR02008,FR09203,FR09203,FR09203,FR82005,FR82005,...,FR04150,FR04150,FR04060,FR04150,FR18080,FR04012,FR18080,FR04002,FR04038,FR04002
3,FR03037,FR03037,FR03037,FR03037,FR02029,FR09303,FR09303,FR09303,FR26114,FR26114,...,FR04060,FR04017,FR04031,FR04060,FR28022,FR04031,FR28022,FR04017,FR04049,FR04017
4,FR02043,FR03032,FR03032,FR03032,FR02007,FR09305,FR09305,FR09305,FR82070,FR82070,...,FR04017,FR04060,FR04179,FR04017,FR18053,FR04071,FR18053,FR04060,FR04331,FR04060
5,FR03032,FR03030,FR02043,FR02043,FR02006,FR09302,FR09302,FR09302,FR82043,FR82043,...,FR04049,FR04031,FR04002,FR04049,FR18079,FR04017,FR18079,FR04031,FR04063,FR04031
6,FR03030,FR02043,FR03030,FR03030,FR02011,FR23152,FR23152,FR23152,FR82040,FR82040,...,FR04012,FR04012,FR04131,FR04012,FR18035,FR04037,FR18035,FR04023,FR04179,FR04023
7,FR02021,FR02021,FR02021,FR02021,FR02013,FR23124,FR23124,FR23124,FR82060,FR82060,...,FR04031,FR04071,FR04118,FR04031,FR06003,FR04118,FR06003,FR04058,FR04150,FR04058
8,FR02029,FR03021,FR02029,FR02029,FR02012,FR23078,FR23078,FR23078,FR82042,FR82042,...,FR04071,FR04131,FR04051,FR04071,FR28115,FR04055,FR28115,FR04131,FR04180,FR04131
9,FR03021,FR02029,FR03021,FR03021,FR02021,FR09304,FR09304,FR09304,FR82041,FR82041,...,FR04118,FR04118,FR04071,FR04118,FR28010,FR04131,FR28010,FR04179,FR04060,FR04179


In [17]:
closest10_stations_df_km

,B001,B002,B003,B004,B005,B006,B007,B008,B009,B010,...,L563,L564,L565,L566,L567,L568,L569,L570,L571,L572
0,2.546476,0.702204,2.106949,1.850120,0.167111,1.657120,0.490297,2.308166,1.053717,1.251146,...,0.562673,6.361774,0.848142,0.562673,32.847896,0.648300,32.847896,4.895682,14.212567,4.895682
1,3.743729,3.810966,4.198500,2.411541,3.680957,3.241721,3.669285,2.777591,2.232074,2.551101,...,10.901557,6.377889,3.026038,10.901557,34.849155,3.858521,34.849155,8.140158,18.983689,8.140158
2,4.948423,4.823265,4.435722,6.996668,5.133624,4.595279,4.835714,5.078185,5.874987,6.047747,...,12.839069,6.659928,5.099630,12.839069,35.120188,3.967788,35.120188,10.027464,19.514071,10.027464
3,12.270026,10.322771,11.857035,10.693344,6.137501,55.135066,54.192815,55.133365,12.805967,13.082888,...,13.577066,7.967987,5.745414,13.577066,35.510676,5.629741,35.510676,10.960105,22.716682,10.960105
4,16.226448,15.081891,16.651681,15.785088,6.500747,56.472010,55.506786,56.484484,29.263873,29.160425,...,14.058418,8.037556,6.447562,14.058418,40.112510,5.710609,40.112510,13.554059,26.966459,13.554059
5,17.015497,17.124324,16.737670,19.227852,6.713874,58.473970,57.989157,58.224378,35.543099,35.621556,...,14.090248,9.673027,6.598578,14.090248,42.384058,6.273663,42.384058,13.780094,29.527949,13.780094
6,17.459932,17.945259,17.683286,19.853263,8.098220,60.999505,59.908775,61.696561,41.813416,41.927320,...,14.948381,10.553089,6.934686,14.948381,45.580568,6.367932,45.580568,14.440741,29.736303,14.440741
7,20.170523,22.095424,20.524455,21.743375,9.768832,64.793648,63.355284,65.243311,42.827367,42.460768,...,15.383270,11.058415,7.160506,15.383270,45.814048,6.629655,45.814048,14.579381,30.350503,14.579381
8,22.511793,24.250644,22.970194,24.912898,10.895315,102.898546,101.461230,103.487120,48.853265,48.971821,...,16.360349,11.099774,7.539168,16.360349,48.715222,6.641291,48.715222,14.630834,31.374725,14.630834
9,24.213915,24.431673,24.526220,26.973590,11.432972,108.366830,107.053567,108.643488,50.938925,51.071327,...,16.791689,11.160772,7.830653,16.791689,51.126074,6.790390,51.126074,14.742451,32.281365,14.742451


## Table détaillée top 10 par station - V1

In [ ]:
identifiant_immeuble_cstb = 'B001' # Mettre identifiant bâtiment CSTB

imm_cstb_desc = df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]
imm_cstb_desc

In [ ]:
df_stations_geodair.head(20)

In [ ]:
# Pas terminé, problème avec :
# substances et concentrations mesurées, période de mesure, indicateur(s) de qualité des mesures, limites de détection et quantification.
cstb_top10stationsGeodair = pd.DataFrame(columns=[
    'CSTB_Code',
    'CSTB_Longitude',
    'CSTB_Latitude',
    'CSTB_Date_debut_enquete',
    'CSTB_Date_fin_enquete',    
    'Geodair_Code',
    'Geodair_Longitude',
    'Geodair_Latitude',
    'Geodair_DistanceToCSTBBuilding',
    'Geodair_Implantation',
    'Geodair_Mobilité'
    ])

for geodair_station_id in closest10_stations_df_code[identifiant_immeuble_cstb]:
    geodair_station_list = []
    
    geodair_station_list.append(identifiant_immeuble_cstb)
    geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Longitude'].iloc[0])
    geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Latitude'].iloc[0])
    geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Date_debut_enquete'].iloc[0])
    geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Date_fin_enquete'].iloc[0])
    
    geodair_station_list.append(geodair_station_id)
    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Longitude'].iloc[0])
    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Latitude'].iloc[0])

    index_station = closest10_stations_df_code.index[closest10_stations_df_code[identifiant_immeuble_cstb]==geodair_station_id][0]
    geodair_station_list.append(closest10_stations_df_km[identifiant_immeuble_cstb].iloc[index_station])
    
    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Implantation'].iloc[0])
    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Mobilité'].iloc[0])
        
    cstb_top10stationsGeodair.loc[len(cstb_top10stationsGeodair)] = geodair_station_list
cstb_top10stationsGeodair


In [ ]:
# Pas terminé, problème avec :
# substances et concentrations mesurées, période de mesure, indicateur(s) de qualité des mesures, limites de détection et quantification.
#cstb_top10stationsGeodair = pd.DataFrame(columns=[
#    'Geodair_Code',
#    'Geodair_Longitude',
#    'Geodair_Latitude',
#    'Geodair_DistanceToCSTBBuilding',
#    'Geodair_Implantation',
#    'Geodair_Mobilité'
#    ])

#for geodair_station_id in closest10_stations_df_code[identifiant_immeuble_cstb]:
#    geodair_station_list = []
#    geodair_station_list.append(geodair_station_id)
#    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Longitude'].iloc[0])
#    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Latitude'].iloc[0])

#    index_station = closest10_stations_df_code.index[closest10_stations_df_code[identifiant_immeuble_cstb]==geodair_station_id][0]
#    geodair_station_list.append(closest10_stations_df_km[identifiant_immeuble_cstb].iloc[index_station])
    
#    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Implantation'].iloc[0])
#    geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Mobilité'].iloc[0])
        
#    cstb_top10stationsGeodair.loc[len(cstb_top10stationsGeodair)] = geodair_station_list
#cstb_top10stationsGeodair


- **Problème 1** : concernant la distinction pollution de fond / de proximité, elle se fait au niveau du polluant, pas de la station. Donc top 10 stations "tout court", pas possible d'obtenir top 3 stations de fond, mais j'ai ajouté info "Implantation". *Réponse: Je mets en pause la notion de station de fond / de proximité (vu qu'elle est au niveau des polluants et pas des stations).*

- **Problème 2** : je suis parvenu à :
    - récupérer les 10 stations Geodair les plus proches de chaque bâtiment CSTB
    - faire la description de chaque bâtiment
    - commencer la table structurée des infos des 10 stations Geodair les plus proches de chaque bâtiment.
> Cependant, nous allons avoir un problème de disponibilité de l'API pour les infos des polluants : il y a 13 polluants et 10 stations Geodair à consulter, il faut donc faire 130 interrogation API pour obtenir toutes les infos pour 1 bâtiment CSTB (il y en a 1106).
**Comment faire ?**
*Réponse* :
> - *Récupération des données sur la période d’enquête d’intérêt, pour le **NO2, PM2.5, PM10 et benzène (C6H6).***
> - *Ensuite, on doit voir comment faire techniquement : télécharger et stocker les fichier 1 fois, avec actualisation quotidienne pour télécharger chaque nouveau fichier ?*

## Obtention données polluants

### Infos

In [ ]:
df_immeubles_cstb2 = df_immeubles_cstb.copy()
df_immeubles_cstb2

In [ ]:
df_immeubles_cstb2['Date_debut_enquete'] = pd.to_datetime(df_immeubles_cstb2['Date_debut_enquete'])

In [ ]:
print(f"Min start : {min(df_immeubles_cstb2['Date_debut_enquete'])}")
print(f"Max start : {max(df_immeubles_cstb2['Date_debut_enquete'])}")
print(f"Min end : {min(df_immeubles_cstb2['Date_fin_enquete'])}")
print(f"Max end : {max(df_immeubles_cstb2['Date_fin_enquete'])}")

### NO2

In [18]:
!cd && cd ../.. && cd Volumes/NO\ NAME/API2Build/data/geodair/NO2 && ls

Export Moy. journalière - NO₂ - 2013-06-15 00_00 - 2014-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2014-01-01 00_00 - 2014-07-01 00_00.csv
Export Moy. journalière - NO₂ - 2014-07-01 00_00 - 2015-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2015-01-01 00_00 - 2015-07-01 00_00.csv
Export Moy. journalière - NO₂ - 2015-07-01 00_00 - 2016-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2016-01-01 00_00 - 2016-07-01 00_00.csv
Export Moy. journalière - NO₂ - 2016-07-01 00_00 - 2017-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2017-01-01 00_00 - 2017-07-01 00_00.csv
Export Moy. journalière - NO₂ - 2017-07-01 00_00 - 2018-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2018-01-01 00_00 - 2018-07-01 00_00.csv
Export Moy. journalière - NO₂ - 2018-07-01 00_00 - 2019-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2019-01-01 00_00 - 2019-07-01 00_00.csv
Export Moy. journalière - NO₂ - 2019-07-01 00_00 - 2020-01-01 00_00.csv
Export Moy. journalière - NO₂ - 2020-01-01 00_00 -

In [19]:
# Sélectionner où se trouvent les fichiers NO2
no2_path = '/Volumes/NO NAME/API2Build/data/geodair/NO2'

In [20]:
# Créer 1 DataFrame par fichier
df1 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2013-06-15 00_00 - 2014-01-01 00_00.csv', sep=";")
df2 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2014-01-01 00_00 - 2014-07-01 00_00.csv', sep=";")
df3 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2014-07-01 00_00 - 2015-01-01 00_00.csv', sep=";")
df4 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2015-01-01 00_00 - 2015-07-01 00_00.csv', sep=";")
df5 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2015-07-01 00_00 - 2016-01-01 00_00.csv', sep=";")
df6 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2016-01-01 00_00 - 2016-07-01 00_00.csv', sep=";")
df7 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2016-07-01 00_00 - 2017-01-01 00_00.csv', sep=";")
df8 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2017-01-01 00_00 - 2017-07-01 00_00.csv', sep=";")
df9 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2017-07-01 00_00 - 2018-01-01 00_00.csv', sep=";")
df10 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2018-01-01 00_00 - 2018-07-01 00_00.csv', sep=";")
df11 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2018-07-01 00_00 - 2019-01-01 00_00.csv', sep=";")
df12 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2019-01-01 00_00 - 2019-07-01 00_00.csv', sep=";")
df13 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2019-07-01 00_00 - 2020-01-01 00_00.csv', sep=";")
df14 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2020-01-01 00_00 - 2020-07-01 00_00.csv', sep=";")
df15 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2020-07-01 00_00 - 2021-01-01 00_00.csv', sep=";")
df16 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2021-01-01 00_00 - 2021-07-01 00_00.csv', sep=";")
df17 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2021-07-01 00_00 - 2022-01-01 00_00.csv', sep=";")
df18 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2022-01-01 00_00 - 2022-07-01 00_00.csv', sep=";")
df19 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2022-07-01 00_00 - 2023-01-01 00_00.csv', sep=";")
df20 = pd.read_csv(f'{no2_path}/Export Moy. journalière - NO₂ - 2023-01-01 00_00 - 2023-03-01 00_00.csv', sep=";")

In [21]:
# Concaténer tous les DataFrames en 1 seul DataFrame
no2_data_full = pd.concat([df1,
                      df2,
                      df3,
                      df4,
                      df5,
                      df6,
                      df7,
                      df8,
                      df9,
                      df10,
                      df11,
                      df12,
                      df13,
                      df14,
                      df15,
                      df16,
                      df17,
                      df18,
                      df19,
                      df20], axis=0)

In [22]:
no2_data_full.head()

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01N20,LORRAINE-ZR,FR01001,Longlaville-Ecole,Périurbaine,NO2,Fond,...,4.7,4.725000,µg-m3,100.0,100,100.0,A,1,49.535004,5.804447
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01N20,LORRAINE-ZR,FR01004,Distroff,Rurale régionale,NO2,Fond,...,0.0,0.000000,µg-m3,0.0,100,0.0,N,-1,49.332497,6.271664
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01005,Hayange,Périurbaine,NO2,Industrielle,...,9.9,9.862500,µg-m3,100.0,100,100.0,R,1,49.324722,6.076108
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01011,Metz-Centre,Urbaine,NO2,Fond,...,13.0,12.745833,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01012,Metz-Borny,Urbaine,NO2,Fond,...,11.0,11.137500,µg-m3,100.0,100,100.0,R,1,49.110283,6.223336


In [23]:
no2_data_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446534 entries, 0 to 22607
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Date de début          1446534 non-null  object 
 1   Date de fin            1446534 non-null  object 
 2   Organisme              1446534 non-null  object 
 3   code zas               1446534 non-null  object 
 4   Zas                    1446534 non-null  object 
 5   code site              1446534 non-null  object 
 6   nom site               1446534 non-null  object 
 7   type d'implantation    1446534 non-null  object 
 8   Polluant               1446534 non-null  object 
 9   type d'influence       1446534 non-null  object 
 10  Réglementaire          1446534 non-null  object 
 11  type d'évaluation      1446534 non-null  object 
 12  type de valeur         1446534 non-null  object 
 13  valeur                 1446534 non-null  float64
 14  valeur brute           14

In [24]:
# Sélectionner les informations essentielles du polluant
no2_data = no2_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [25]:
no2_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01001,Longlaville-Ecole,Périurbaine,NO2,Fond,mesures fixes,4.725000,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01004,Distroff,Rurale régionale,NO2,Fond,mesures fixes,0.000000,µg-m3,N
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01005,Hayange,Périurbaine,NO2,Industrielle,mesures fixes,9.862500,µg-m3,R
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01011,Metz-Centre,Urbaine,NO2,Fond,mesures fixes,12.745833,µg-m3,A
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01012,Metz-Borny,Urbaine,NO2,Fond,mesures fixes,11.137500,µg-m3,R
...,...,...,...,...,...,...,...,...,...,...,...,...
22603,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82040,Dole centre,Urbaine,NO2,Fond,mesures fixes,11.025000,µg-m3,A
22604,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82041,Tavaux,Rurale près des villes,NO2,Industrielle,mesures fixes,8.504167,µg-m3,A
22605,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82042,Damparis,Rurale près des villes,NO2,Industrielle,mesures fixes,9.341667,µg-m3,R
22606,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82060,Vesoul Pres Caillet,Urbaine,NO2,Fond,mesures fixes,15.404167,µg-m3,A


In [26]:
# Convertir les dates en datetime
no2_data['Date de début'] = pd.to_datetime(no2_data['Date de début'])
no2_data['Date de fin'] = pd.to_datetime(no2_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/2136219898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no2_data['Date de début'] = pd.to_datetime(no2_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/2136219898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no2_data['Date de fin'] = pd.to_datetime(no2_data['Date de fin'])


In [27]:
no2_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446534 entries, 0 to 22607
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Date de début        1446534 non-null  datetime64[ns]
 1   Date de fin          1446534 non-null  datetime64[ns]
 2   Organisme            1446534 non-null  object        
 3   code site            1446534 non-null  object        
 4   nom site             1446534 non-null  object        
 5   type d'implantation  1446534 non-null  object        
 6   Polluant             1446534 non-null  object        
 7   type d'influence     1446534 non-null  object        
 8   type d'évaluation    1446534 non-null  object        
 9   valeur brute         1446534 non-null  float64       
 10  unité de mesure      1446534 non-null  object        
 11  code qualité         1446534 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 143.5

In [ ]:
# Exporter le DataFrame si besoin
no2_data.to_csv(f'/Volumes/NO NAME/API2Build/data/geodair/no2_data.csv')

### PM 2.5

In [28]:
!cd && cd ../.. && cd Volumes/NO\ NAME/API2Build/data/geodair/PM25 && ls

Export Moy. journalière - PM₂.₅ - 2013-06-15 00_00 - 2014-01-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2014-01-01 00_00 - 2014-07-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2014-07-01 00_00 - 2015-01-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2015-01-01 00_00 - 2015-07-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2015-07-01 00_00 - 2016-01-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2016-01-01 00_00 - 2016-07-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2016-07-01 00_00 - 2017-01-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2017-01-01 00_00 - 2017-07-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2017-07-01 00_00 - 2018-01-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2018-01-01 00_00 - 2018-07-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2018-07-01 00_00 - 2019-01-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2019-01-01 00_00 - 2019-07-01 00_00.csv
Export Moy. journalière - PM₂.₅ - 2019-07-01 00_00 - 2020-01-01 00_00.csv
Export Moy. journalière 

In [29]:
# Sélectionner où se trouvent les fichiers PM25
pm25_path = '/Volumes/NO NAME/API2Build/data/geodair/PM25'

In [30]:
# Créer 1 DataFrame par fichier
df1 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2013-06-15 00_00 - 2014-01-01 00_00.csv', sep=";")
df2 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2014-01-01 00_00 - 2014-07-01 00_00.csv', sep=";")
df3 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2014-07-01 00_00 - 2015-01-01 00_00.csv', sep=";")
df4 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2015-01-01 00_00 - 2015-07-01 00_00.csv', sep=";")
df5 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2015-07-01 00_00 - 2016-01-01 00_00.csv', sep=";")
df6 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2016-01-01 00_00 - 2016-07-01 00_00.csv', sep=";")
df7 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2016-07-01 00_00 - 2017-01-01 00_00.csv', sep=";")
df8 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2017-01-01 00_00 - 2017-07-01 00_00.csv', sep=";")
df9 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2017-07-01 00_00 - 2018-01-01 00_00.csv', sep=";")
df10 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2018-01-01 00_00 - 2018-07-01 00_00.csv', sep=";")
df11 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2018-07-01 00_00 - 2019-01-01 00_00.csv', sep=";")
df12 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2019-01-01 00_00 - 2019-07-01 00_00.csv', sep=";")
df13 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2019-07-01 00_00 - 2020-01-01 00_00.csv', sep=";")
df14 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2020-01-01 00_00 - 2020-07-01 00_00.csv', sep=";")
df15 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2020-07-01 00_00 - 2021-01-01 00_00.csv', sep=";")
df16 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2021-01-01 00_00 - 2021-07-01 00_00.csv', sep=";")
df17 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2021-07-01 00_00 - 2022-01-01 00_00.csv', sep=";")
df18 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2022-01-01 00_00 - 2022-07-01 00_00.csv', sep=";")
df19 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2022-07-01 00_00 - 2023-01-01 00_00.csv', sep=";")
df20 = pd.read_csv(f'{pm25_path}/Export Moy. journalière - PM₂.₅ - 2023-01-01 00_00 - 2023-03-01 00_00.csv', sep=";")

In [31]:
# Concaténer tous les DataFrames en 1 seul DataFrame
pm25_data_full = pd.concat([df1,
                      df2,
                      df3,
                      df4,
                      df5,
                      df6,
                      df7,
                      df8,
                      df9,
                      df10,
                      df11,
                      df12,
                      df13,
                      df14,
                      df15,
                      df16,
                      df17,
                      df18,
                      df19,
                      df20], axis=0)

In [32]:
pm25_data_full.head()

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01011,Metz-Centre,Urbaine,PM2.5,Fond,...,8.9,8.887500,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01020,Thionville-Centre,Urbaine,PM2.5,Fond,...,7.9,7.887500,µg-m3,100.0,100,100.0,A,1,49.358337,6.156942
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02020,Rognac les Brets,Périurbaine,PM2.5,Fond,...,15.0,15.178261,µg-m3,96.0,100,96.0,R,1,43.510802,5.220046
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03006,MARSEILLE RABATAU,Urbaine,PM2.5,Trafic,...,16.0,16.100000,µg-m3,96.0,100,96.0,R,1,43.276451,5.397360
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR03A02,PROVENCE-ALPES-COTE-D-AZUR-MARSEILLE,FR03029,AIX CENTRE ECOLE ART,Urbaine,PM2.5,Fond,...,15.0,14.810000,µg-m3,83.0,100,83.0,R,1,43.530285,5.440912


In [33]:
# Sélectionner les informations essentielles du polluant
pm25_data = pm25_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [34]:
pm25_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01011,Metz-Centre,Urbaine,PM2.5,Fond,mesures fixes,8.887500,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01020,Thionville-Centre,Urbaine,PM2.5,Fond,mesures fixes,7.887500,µg-m3,A
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02020,Rognac les Brets,Périurbaine,PM2.5,Fond,mesures fixes,15.178261,µg-m3,R
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR03006,MARSEILLE RABATAU,Urbaine,PM2.5,Trafic,mesures fixes,16.100000,µg-m3,R
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR03029,AIX CENTRE ECOLE ART,Urbaine,PM2.5,Fond,mesures fixes,14.810000,µg-m3,R
...,...,...,...,...,...,...,...,...,...,...,...,...
13116,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82006,Besancon Prevoyance,Urbaine,PM2.5,Fond,mesures fixes,12.541667,µg-m3,A
13117,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82010,Montbéliard centre,Urbaine,PM2.5,Fond,mesures fixes,10.837500,µg-m3,A
13118,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82050,Lons-le-Saunier CV,Urbaine,PM2.5,Fond,mesures fixes,13.225000,µg-m3,A
13119,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82060,Vesoul Pres Caillet,Urbaine,PM2.5,Fond,mesures fixes,13.362500,µg-m3,A


In [35]:
# Convertir les dates en datetime
pm25_data['Date de début'] = pd.to_datetime(pm25_data['Date de début'])
pm25_data['Date de fin'] = pd.to_datetime(pm25_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/1135404810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm25_data['Date de début'] = pd.to_datetime(pm25_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/1135404810.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm25_data['Date de fin'] = pd.to_datetime(pm25_data['Date de fin'])


In [36]:
pm25_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568412 entries, 0 to 13120
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date de début        568412 non-null  datetime64[ns]
 1   Date de fin          568412 non-null  datetime64[ns]
 2   Organisme            568412 non-null  object        
 3   code site            568412 non-null  object        
 4   nom site             568412 non-null  object        
 5   type d'implantation  568412 non-null  object        
 6   Polluant             568412 non-null  object        
 7   type d'influence     568412 non-null  object        
 8   type d'évaluation    568412 non-null  object        
 9   valeur brute         568412 non-null  float64       
 10  unité de mesure      568412 non-null  object        
 11  code qualité         568412 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 56.4+ MB


In [ ]:
# Exporter le DataFrame si besoin
pm25_data.to_csv(f'/Volumes/NO NAME/API2Build/data/geodair/pm25_data.csv')

### PM 10

In [37]:
!cd && cd ../.. && cd Volumes/NO\ NAME/API2Build/data/geodair/PM10 && ls

Export Moy. journalière - PM₁₀ - 2013-06-15 00_00 - 2014-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2014-01-01 00_00 - 2014-07-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2014-07-01 00_00 - 2015-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2015-01-01 00_00 - 2015-07-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2015-07-01 00_00 - 2016-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2016-01-01 00_00 - 2016-07-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2016-07-01 00_00 - 2017-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2017-01-01 00_00 - 2017-07-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2017-07-01 00_00 - 2018-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2018-01-01 00_00 - 2018-07-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2018-07-01 00_00 - 2019-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2019-01-01 00_00 - 2019-07-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2019-07-01 00_00 - 2020-01-01 00_00.csv
Export Moy. journalière - PM₁₀ - 2020

In [38]:
# Sélectionner où se trouvent les fichiers PM10
pm10_path = '/Volumes/NO NAME/API2Build/data/geodair/PM10'

In [39]:
# Créer 1 DataFrame par fichier
df1 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2013-06-15 00_00 - 2014-01-01 00_00.csv', sep=";")
df2 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2014-01-01 00_00 - 2014-07-01 00_00.csv', sep=";")
df3 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2014-07-01 00_00 - 2015-01-01 00_00.csv', sep=";")
df4 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2015-01-01 00_00 - 2015-07-01 00_00.csv', sep=";")
df5 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2015-07-01 00_00 - 2016-01-01 00_00.csv', sep=";")
df6 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2016-01-01 00_00 - 2016-07-01 00_00.csv', sep=";")
df7 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2016-07-01 00_00 - 2017-01-01 00_00.csv', sep=";")
df8 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2017-01-01 00_00 - 2017-07-01 00_00.csv', sep=";")
df9 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2017-07-01 00_00 - 2018-01-01 00_00.csv', sep=";")
df10 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2018-01-01 00_00 - 2018-07-01 00_00.csv', sep=";")
df11 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2018-07-01 00_00 - 2019-01-01 00_00.csv', sep=";")
df12 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2019-01-01 00_00 - 2019-07-01 00_00.csv', sep=";")
df13 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2019-07-01 00_00 - 2020-01-01 00_00.csv', sep=";")
df14 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2020-01-01 00_00 - 2020-07-01 00_00.csv', sep=";")
df15 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2020-07-01 00_00 - 2021-01-01 00_00.csv', sep=";")
df16 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2021-01-01 00_00 - 2021-07-01 00_00.csv', sep=";")
df17 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2021-07-01 00_00 - 2022-01-01 00_00.csv', sep=";")
df18 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2022-01-01 00_00 - 2022-07-01 00_00.csv', sep=";")
df19 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2022-07-01 00_00 - 2023-01-01 00_00.csv', sep=";")
df20 = pd.read_csv(f'{pm10_path}/Export Moy. journalière - PM₁₀ - 2023-01-01 00_00 - 2023-03-01 00_00.csv', sep=";")

In [40]:
# Concaténer tous les DataFrames en 1 seul DataFrame
pm10_data_full = pd.concat([df1,
                      df2,
                      df3,
                      df4,
                      df5,
                      df6,
                      df7,
                      df8,
                      df9,
                      df10,
                      df11,
                      df12,
                      df13,
                      df14,
                      df15,
                      df16,
                      df17,
                      df18,
                      df19,
                      df20], axis=0)

In [41]:
pm10_data_full.head()

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01N20,LORRAINE-ZR,FR01001,Longlaville-Ecole,Périurbaine,PM10,Fond,...,8.0,7.537500,µg-m3,100.0,100,100.0,A,1,49.535004,5.804447
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01005,Hayange,Périurbaine,PM10,Industrielle,...,10.0,9.962500,µg-m3,100.0,100,100.0,A,1,49.324722,6.076108
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01006,Florange,Périurbaine,PM10,Fond,...,0.0,0.000000,µg-m3,0.0,100,0.0,N,-1,49.320831,6.140000
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01009,Gandrange,Périurbaine,PM10,Industrielle,...,11.0,10.554167,µg-m3,100.0,100,100.0,A,1,49.271393,6.132500
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01A01,LORRAINE-METZ,FR01011,Metz-Centre,Urbaine,PM10,Fond,...,9.0,9.387500,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833


In [42]:
# Sélectionner les informations essentielles du polluant
pm10_data = pm10_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [43]:
pm10_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01001,Longlaville-Ecole,Périurbaine,PM10,Fond,mesures fixes,7.537500,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01005,Hayange,Périurbaine,PM10,Industrielle,mesures fixes,9.962500,µg-m3,A
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01006,Florange,Périurbaine,PM10,Fond,mesures fixes,0.000000,µg-m3,N
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01009,Gandrange,Périurbaine,PM10,Industrielle,mesures fixes,10.554167,µg-m3,A
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO GRAND EST,FR01011,Metz-Centre,Urbaine,PM10,Fond,mesures fixes,9.387500,µg-m3,A
...,...,...,...,...,...,...,...,...,...,...,...,...
20318,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82030,Montandon Baresans,Rurale nationale,PM10,Fond,mesures fixes,13.825000,µg-m3,A
20319,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82040,Dole centre,Urbaine,PM10,Fond,mesures fixes,13.783333,µg-m3,A
20320,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82043,Chatenois,Périurbaine,PM10,Industrielle,mesures fixes,12.458333,µg-m3,A
20321,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO BOURGOGNE-FRANCHE-COMTE,FR82060,Vesoul Pres Caillet,Urbaine,PM10,Fond,mesures fixes,21.333333,µg-m3,A


In [44]:
# Convertir les dates en datetime
pm10_data['Date de début'] = pd.to_datetime(pm10_data['Date de début'])
pm10_data['Date de fin'] = pd.to_datetime(pm10_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/1290666400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm10_data['Date de début'] = pd.to_datetime(pm10_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/1290666400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm10_data['Date de fin'] = pd.to_datetime(pm10_data['Date de fin'])


In [45]:
pm10_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1303076 entries, 0 to 20322
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Date de début        1303076 non-null  datetime64[ns]
 1   Date de fin          1303076 non-null  datetime64[ns]
 2   Organisme            1303076 non-null  object        
 3   code site            1303076 non-null  object        
 4   nom site             1303076 non-null  object        
 5   type d'implantation  1303076 non-null  object        
 6   Polluant             1303076 non-null  object        
 7   type d'influence     1303076 non-null  object        
 8   type d'évaluation    1303076 non-null  object        
 9   valeur brute         1303076 non-null  float64       
 10  unité de mesure      1303076 non-null  object        
 11  code qualité         1303076 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 129.2

In [ ]:
# Exporter le DataFrame si besoin
pm10_data.to_csv(f'/Volumes/NO NAME/API2Build/data/geodair/pm10_data.csv')

### C6H6

In [46]:
!cd && cd ../.. && cd Volumes/NO\ NAME/API2Build/data/geodair/C6H6 && ls

Export Moy. journalière - C₆H₆ - 2013-06-15 00_00 - 2023-03-01 00_00.csv


In [47]:
# Sélectionner où se trouvent les fichiers C6H6
c6h6_path = '/Volumes/NO NAME/API2Build/data/geodair/C6H6'

In [48]:
# Créer 1 DataFrame par fichier
c6h6_data_full = pd.read_csv(f'{c6h6_path}/Export Moy. journalière - C₆H₆ - 2013-06-15 00_00 - 2023-03-01 00_00.csv', sep=";")

In [49]:
c6h6_data_full.head()

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02001,Berre l'Etang,Périurbaine,C6H6,Industrielle,...,0.56,0.564348,µg-m3,96.0,100,96.0,A,1,43.486234,5.171939
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02N30,PROVENCE-ALPES-COTE-D-AZUR-ZI,FR02028,Martigues Lavera,Périurbaine,C6H6,Industrielle,...,0.50,0.503750,µg-m3,100.0,100,100.0,A,1,43.386564,5.026868
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR07A01,AUVERGNE-CLERMONT,FR07034,Clermont-Esplanade Gare,Urbaine,C6H6,Trafic,...,0.71,0.711667,µg-m3,100.0,100,100.0,A,1,45.775696,3.096250
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR11N10,NORD-PAS-DE-CALAIS-ZUR,FR10012,Mardyck,Périurbaine,C6H6,Industrielle,...,0.18,0.175000,µg-m3,100.0,100,100.0,A,1,51.019430,2.250990
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR11A01,NORD-PAS-DE-CALAIS-LILLE,FR11034,Roubaix Serres,Urbaine,C6H6,Trafic,...,0.18,0.181250,µg-m3,100.0,100,100.0,A,1,50.706535,3.180660


In [50]:
# Sélectionner les informations essentielles du polluant
c6h6_data = c6h6_data_full[[
    "Date de début",
    "Date de fin",
    "Organisme",
    "code site",
    "nom site",
    "type d'implantation",
    "Polluant",
    "type d'influence",
    "type d'évaluation",
    "valeur brute",
    "unité de mesure",
    "code qualité"]]

In [51]:
c6h6_data

,Date de début,Date de fin,Organisme,code site,nom site,type d'implantation,Polluant,type d'influence,type d'évaluation,valeur brute,unité de mesure,code qualité
0,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02001,Berre l'Etang,Périurbaine,C6H6,Industrielle,mesures fixes,0.564348,µg-m3,A
1,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO SUD,FR02028,Martigues Lavera,Périurbaine,C6H6,Industrielle,mesures fixes,0.503750,µg-m3,A
2,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR07034,Clermont-Esplanade Gare,Urbaine,C6H6,Trafic,mesures fixes,0.711667,µg-m3,A
3,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR10012,Mardyck,Périurbaine,C6H6,Industrielle,mesures fixes,0.175000,µg-m3,A
4,2013/06/15 00:00:00,2013/06/15 23:59:59,ATMO HAUTS DE FRANCE,FR11034,Roubaix Serres,Urbaine,C6H6,Trafic,mesures fixes,0.181250,µg-m3,A
...,...,...,...,...,...,...,...,...,...,...,...,...
38658,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO HAUTS DE FRANCE,FR10012,Mardyck,Périurbaine,C6H6,Industrielle,estimation objective,1.354167,µg-m3,A
38659,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO HAUTS DE FRANCE,FR11007,Lille Leeds,Urbaine,C6H6,Trafic,estimation objective,0.727500,µg-m3,A
38660,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR20029,FEYZIN STADE,Périurbaine,C6H6,Industrielle,estimation objective,2.200417,µg-m3,A
38661,2023/02/28 00:00:00,2023/02/28 23:59:59,ATMO AUVERGNE-RHÔNE-ALPES,FR20064,VERNAISON,Périurbaine,C6H6,Industrielle,estimation objective,0.832500,µg-m3,A


In [52]:
# Convertir les dates en datetime
c6h6_data['Date de début'] = pd.to_datetime(c6h6_data['Date de début'])
c6h6_data['Date de fin'] = pd.to_datetime(c6h6_data['Date de fin'])

/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/3282342663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c6h6_data['Date de début'] = pd.to_datetime(c6h6_data['Date de début'])
/var/folders/1j/7bkgcqrn02n90wb3kn4sf4ww0000gn/T/ipykernel_23586/3282342663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c6h6_data['Date de fin'] = pd.to_datetime(c6h6_data['Date de fin'])


In [53]:
c6h6_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38663 entries, 0 to 38662
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date de début        38663 non-null  datetime64[ns]
 1   Date de fin          38663 non-null  datetime64[ns]
 2   Organisme            38663 non-null  object        
 3   code site            38663 non-null  object        
 4   nom site             38663 non-null  object        
 5   type d'implantation  38663 non-null  object        
 6   Polluant             38663 non-null  object        
 7   type d'influence     38663 non-null  object        
 8   type d'évaluation    38663 non-null  object        
 9   valeur brute         38663 non-null  float64       
 10  unité de mesure      38663 non-null  object        
 11  code qualité         38663 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 3.5+ MB


In [ ]:
# Exporter le DataFrame si besoin
c6h6_data.to_csv(f'/Volumes/NO NAME/API2Build/data/geodair/c6h6_data.csv')

## Table détaillée top 10 par station - V2

### Draft

In [ ]:
df2 = pd.DataFrame(df_immeubles_cstb['Date_fin_enquete']-df_immeubles_cstb['Date_debut_enquete'])
df2.value_counts()

In [ ]:
cstb_top10stationsGeodair

In [ ]:
row = df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]
row

In [ ]:
start_date = row['Date_debut_enquete'].values[0]
end_date = row['Date_fin_enquete'].values[0]

In [ ]:
if pd.isnull(end_date):
#    end_date = start_date+np.timedelta64(1,'D')
    end_date = start_date
    
    dates = pd.date_range(start_date,end_date,freq='D')

else:
    dates = pd.date_range(start_date,end_date,freq='D')
    
dates

In [ ]:
no2_data[(no2_data['Date de début']==dates[0]) & (no2_data['code site']== 'FR03043')]

In [ ]:
no2_data[(no2_data['Date de début']==dates[0]) & (no2_data['code site']== 'FR03043')]['valeur brute'].values[0]

In [ ]:
#for _, immeuble in df_immeubles_cstb.iterrows():
#    start_date = immeuble['Date_debut_enquete'].values[0]
#    end_date = immeuble['Date_fin_enquete'].values[0]
#    
#    if pd.isnull(end_date):
#        #end_date = start_date+np.timedelta64(1,'D')
#        end_date = start_date
#
#        dates = pd.date_range(start_date,end_date,freq='D')
#
#    else:
#        dates = pd.date_range(start_date,end_date,freq='D')
#    
#dates

In [ ]:
cstb_top10stationsGeodair['polluant']='NO2'

for date in dates :
    pollutant_values=[]
    
    for _, station in cstb_top10stationsGeodair.iterrows():
        Geodair_Code = station['Geodair_Code'] 
        
        if len(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==Geodair_Code)])==0:
            pollutant_values.append(np.nan)
        else:
            pollutant_values.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==Geodair_Code)]['valeur brute'].values[0])
            
    cstb_top10stationsGeodair[date]=pollutant_values


In [ ]:
cstb_top10stationsGeodair

### Finalisation V2

In [ ]:
# Sélection des dates de la période d'enquête pour le bâtiment CSTB sélectionné
row = df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]

start_date = row['Date_debut_enquete'].values[0]
end_date = row['Date_fin_enquete'].values[0]

# Si date_fin_enquete = 'NaT', cela signifie que la période d'enquête était de 1 jour
if pd.isnull(end_date):
    end_date = start_date

# Création de la plage de dates de la période d'enquête pour le bâtiment CSTB sélectionné
dates = pd.date_range(start_date,end_date,freq='D')

# NO2
# NO2 - Ajout de l'information du polluant sélectionné dans le DataFrame
cstb_top10stationsGeodair['polluant 1']='NO2'

# NO2 - Intégration des valeurs du polluant sélectionné, pour chaque date de la période d'enquête
for date in dates :
    pollutant_values=[]
    
    for _, station in cstb_top10stationsGeodair.iterrows():
        Geodair_Code = station['Geodair_Code'] 
        
        if len(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==Geodair_Code)])==0:
            pollutant_values.append(np.nan)
        else:
            pollutant_values.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==Geodair_Code)]['valeur brute'].values[0])
            
    cstb_top10stationsGeodair[f'NO2 - {date}']=pollutant_values

# PM2.5
# PM2.5 - Ajout de l'information du polluant sélectionné dans le DataFrame
cstb_top10stationsGeodair['polluant 2']='PM2.5'

# PM2.5 - Intégration des valeurs du polluant sélectionné, pour chaque date de la période d'enquête
for date in dates :
    pollutant_values=[]
    
    for _, station in cstb_top10stationsGeodair.iterrows():
        Geodair_Code = station['Geodair_Code'] 
        
        if len(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==Geodair_Code)])==0:
            pollutant_values.append(np.nan)
        else:
            pollutant_values.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==Geodair_Code)]['valeur brute'].values[0])
            
    cstb_top10stationsGeodair[f'PM2.5 - {date}']=pollutant_values
    
# PM10
# PM10 - Ajout de l'information du polluant sélectionné dans le DataFrame
cstb_top10stationsGeodair['polluant 3']='PM10'

# PM10 - Intégration des valeurs du polluant sélectionné, pour chaque date de la période d'enquête
for date in dates :
    pollutant_values=[]
    
    for _, station in cstb_top10stationsGeodair.iterrows():
        Geodair_Code = station['Geodair_Code'] 
        
        if len(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==Geodair_Code)])==0:
            pollutant_values.append(np.nan)
        else:
            pollutant_values.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==Geodair_Code)]['valeur brute'].values[0])
            
    cstb_top10stationsGeodair[f'PM10 - {date}']=pollutant_values

# C6H6
# C6H6 - Ajout de l'information du polluant sélectionné dans le DataFrame
cstb_top10stationsGeodair['polluant 4']='C6H6'

# C6H6 - Intégration des valeurs du polluant sélectionné, pour chaque date de la période d'enquête
for date in dates :
    pollutant_values=[]
    
    for _, station in cstb_top10stationsGeodair.iterrows():
        Geodair_Code = station['Geodair_Code'] 
        
        if len(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==Geodair_Code)])==0:
            pollutant_values.append(np.nan)
        else:
            pollutant_values.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==Geodair_Code)]['valeur brute'].values[0])
            
    cstb_top10stationsGeodair[f'C6H6 - {date}']=pollutant_values

# Ajout de l'unité de mesure
cstb_top10stationsGeodair['unité de mesure']='µg-m3'

cstb_top10stationsGeodair


### Finalisation V3

In [54]:
# Sélection du bâtiment CSTB à étudier
identifiant_immeuble_cstb = 'L568' # Mettre identifiant bâtiment CSTB

# Sélection des polluants à ajouter dans la table
pollutants = ['NO2', 'PM2.5', 'PM10', 'C6H6']

# Sélection des dates de la période d'enquête pour le bâtiment CSTB sélectionné
row = df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]

start_date = row['Date_debut_enquete'].values[0]
end_date = row['Date_fin_enquete'].values[0]

# Si date_fin_enquete = 'NaT', cela signifie que la période d'enquête était de 1 jour
if pd.isnull(end_date):
    end_date = start_date

# Création de la plage de dates de la période d'enquête pour le bâtiment CSTB sélectionné
dates = pd.date_range(start_date,end_date,freq='D')

# Création de la table pour le bâtiment CSTB sélectionnné
cstb_top10stationsGeodair = pd.DataFrame(columns=[
    'CSTB_Code',
    'CSTB_Longitude',
    'CSTB_Latitude',
    'CSTB_Date_debut_enquete',
    'CSTB_Date_fin_enquete',    
    'Geodair_Code',
    'Geodair_Longitude',
    'Geodair_Latitude',
    'Geodair_DistanceToCSTBBuilding',
    'Geodair_Implantation',
    'Geodair_Mobilité',
    'Geodair_Polluant',
    'Geodair_Date',
    'Geodair_Valeur_brute',
    'Geodair_Unité_mesure',
    'Geodair_Code_qualité'
    ])

for geodair_station_id in closest10_stations_df_code[identifiant_immeuble_cstb]:
    
    for pollutant in pollutants:
        
        for date in dates:
    
            # Création liste vide pour chaque station Geod'air
            geodair_station_list = []
            
            # Ajout informations sur le bâtiment CSTB
            geodair_station_list.append(identifiant_immeuble_cstb)
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Longitude'].iloc[0])
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Latitude'].iloc[0])
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Date_debut_enquete'].iloc[0])
            geodair_station_list.append(df_immeubles_cstb[df_immeubles_cstb['Identifiant']==identifiant_immeuble_cstb]['Date_fin_enquete'].iloc[0])

            # Ajout informations sur la station Geod'air
            geodair_station_list.append(geodair_station_id)
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Longitude'].iloc[0])
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Latitude'].iloc[0])

            # Ajout de la distance du bâtiment CSTB à la station Geod'air sélectionnée
            index_station = closest10_stations_df_code.index[closest10_stations_df_code[identifiant_immeuble_cstb]==geodair_station_id][0]
            geodair_station_list.append(closest10_stations_df_km[identifiant_immeuble_cstb].iloc[index_station])

            # Ajout informations complémentaires sur la station Geod'air
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Implantation'].iloc[0])
            geodair_station_list.append(df_stations_geodair[df_stations_geodair['Code']==geodair_station_id]['Mobilité'].iloc[0])

            # Ajout information sur le polluant sélectionné
            geodair_station_list.append(pollutant)
            
            # Ajout date sélectionnée dans la période d'enquête pour le bâtiment CSTB
            geodair_station_list.append(date)
            
            # Ajout informations sur le polluant sélectionné à la date sélectionnée
            if pollutant == 'NO2':            
                if len(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(no2_data[(no2_data['Date de début']==date) & (no2_data['code site']==geodair_station_id)]['code qualité'].values[0])

            elif pollutant == 'PM2.5':            
                if len(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(pm25_data[(pm25_data['Date de début']==date) & (pm25_data['code site']==geodair_station_id)]['code qualité'].values[0])
                    
            elif pollutant == 'PM10':            
                if len(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(pm10_data[(pm10_data['Date de début']==date) & (pm10_data['code site']==geodair_station_id)]['code qualité'].values[0])

            elif pollutant == 'C6H6':            
                if len(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)])==0:
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                    geodair_station_list.append(np.nan)
                else:
                    geodair_station_list.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)]['valeur brute'].values[0])
                    geodair_station_list.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)]['unité de mesure'].values[0])
                    geodair_station_list.append(c6h6_data[(c6h6_data['Date de début']==date) & (c6h6_data['code site']==geodair_station_id)]['code qualité'].values[0])

            # Ajout de la liste complète au DataFrame dans une nouvelle ligne
            cstb_top10stationsGeodair.loc[len(cstb_top10stationsGeodair)] = geodair_station_list
        
cstb_top10stationsGeodair


,CSTB_Code,CSTB_Longitude,CSTB_Latitude,CSTB_Date_debut_enquete,CSTB_Date_fin_enquete,Geodair_Code,Geodair_Longitude,Geodair_Latitude,Geodair_DistanceToCSTBBuilding,Geodair_Implantation,Geodair_Mobilité,Geodair_Polluant,Geodair_Date,Geodair_Valeur_brute,Geodair_Unité_mesure,Geodair_Code_qualité
0,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04179,2.269889,48.830387,0.64830,Urbaine,Station Fixe,NO2,2022-11-08,12.029167,µg-m3,A
1,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04179,2.269889,48.830387,0.64830,Urbaine,Station Fixe,NO2,2022-11-09,16.475000,µg-m3,A
2,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04179,2.269889,48.830387,0.64830,Urbaine,Station Fixe,NO2,2022-11-10,20.900000,µg-m3,A
3,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04179,2.269889,48.830387,0.64830,Urbaine,Station Fixe,NO2,2022-11-11,17.941667,µg-m3,A
4,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04179,2.269889,48.830387,0.64830,Urbaine,Station Fixe,NO2,2022-11-12,28.108333,µg-m3,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04131,2.330278,48.873333,6.79039,Urbaine,Station Fixe,C6H6,2022-11-11,NaN,NaN,NaN
316,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04131,2.330278,48.873333,6.79039,Urbaine,Station Fixe,C6H6,2022-11-12,NaN,NaN,NaN
317,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04131,2.330278,48.873333,6.79039,Urbaine,Station Fixe,C6H6,2022-11-13,NaN,NaN,NaN
318,L568,2.273457,48.825051,2022-11-08,2022-11-15,FR04131,2.330278,48.873333,6.79039,Urbaine,Station Fixe,C6H6,2022-11-14,NaN,NaN,NaN


### Export

In [ ]:
# Exporter le DataFrame si besoin
cstb_top10stationsGeodair.to_csv(f'/Users/llm/code/TheLab75/glab/api2build/data/geodair/{identifiant_immeuble_cstb}_top10Geodair.csv')